<a href="https://colab.research.google.com/github/htcysl/PCA-on-RGB-nDSM/blob/main/principle_component_analysis_on_orthofoto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**rasterio** library for satellite imagery and terrain models

 Rasterio reads and writes these formats and provides a Python API based on Numpy N-dimensional arrays and GeoJSON.

In [ ]:
!pip install -q rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 50.0 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/4.sınıfBahar/MipMap/DATASET/Alula/
%ls

/content/drive/MyDrive/4.sınıfBahar/MipMap/DATASET/Alula
 10cm/       5cm/              'Masks(Original)'/        ReducedData/    Tile_20cm/
 15cm/       CombinedData/      nDSM/                    ReducedOrtho/   Tile_30cm/
 20cm/       datasetInfo.gdoc   nDSM_normalized/         TextOutput/     Tile_5cm/
 20cm.txt/   DSM/               Ortho/                   Tile_10cm/
 30cm/       DTM/               path_to_save_image.jpg   Tile_15cm/


In [ ]:
import numpy as np
from osgeo import gdal
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import tifffile as tiff

In [ ]:
file_list = ['A1','A2','B1','B2','B3','B4','C1','C2','C3','C4','D1','D2','D3','D4']
for file in file_list:

   dataset = gdal.Open(f'Ortho/Ortho_{file}.tif')

   # Extract the channels
   channels = [dataset.GetRasterBand(i+1).ReadAsArray() for i in range(dataset.RasterCount)]
   data = np.stack(channels, axis=-1)
   dataset = None
   channels = None

   # Reshape data to 2D array (pixels x channels)
   height, width, num_channels = data.shape
   data_reshaped = data.reshape(-1, num_channels)
   data = None

   # Standardize the data
   scaler = StandardScaler()
   data_scaled = scaler.fit_transform(data_reshaped)
   data_reshaped = None
   scaler = None

   # Apply PCA to reduce the number of channels
   pca = PCA(n_components=2)
   data_pca = pca.fit_transform(data_scaled)
   data_scaled = None
   pca = None

    # Reshape back to the original dimensions (height x width x reduced channels)
   data_pca_reshaped = data_pca.reshape(height, width, -1)
   data_pca = None

    # Save the transformed data to a new 3-channel TIFF file
   output_tif = f'ReducedOrtho/ReducedOrtho_{file}'


   tiff.imwrite(output_tif, data_pca_reshaped.astype(np.float32))

   print(f'Saved 3-channel TIFF file to {output_tif}')
   data_pca_reshaped = None
